# datAcron Ontology Traversals

This Jupyter NB serves as validation for the example queries and additionally as navigation and exploration notebook for the datAcron triple store. Where data is saved to CSVs, these CSV files are available in the data/ folder.


# Definition of a query function

First, we define a query function that connects to the RDF store and returns the query results. There are two triple stores available: 
 - http://83.212.239.107:8890/sparql : the big triple store containing all data
 - http://83.212.239.109:3434/sparql : the centralized sandbox triple store
 
In general, I connect to the big store, because we will see later that the 109er store will yield no results in most cases.

In [1]:
import pandas as pd
from pandas.io.json import json_normalize, read_json
from SPARQLWrapper import SPARQLWrapper, JSON

def query_data(sparql_query, sparql_service_url='http://83.212.239.107:8890/sparql'):
    """
    Query the endpoint with the given query string and return the results as a pandas Dataframe.
    """
    # create the connection to the endpoint; set return format; ask for result
    sparql = SPARQLWrapper(sparql_service_url)  
    sparql.setQuery(sparql_query)
    sparql.setReturnFormat(JSON)
    result = sparql.query().convert()
    
    #clean up the column mess (thanks to David Knodt)
    for row in result['results']['bindings']:
        for key in row.keys():
            row[key] = row[key]['value']            
    if len(result["results"]["bindings"]):
        #return json_normalize(result["results"]["bindings"])
        #return read_json(str(result["results"]["bindings"]), orient = 'records' )
        dftemp = pd.DataFrame(result["results"]["bindings"])   #Problem: Pandas DataFrame() method sorted cols alphabetically
        dftemp = dftemp[result['head']['vars']]                #Solution: this reorders colmns like they were returned from SPARQL
        return dftemp
    else:
        return pd.DataFrame(columns=(result['head']['vars']))
    

    

In [2]:
def query_value_cleaning(df):
    """
    Drop URI prefixes from result values. Excpects a Pandas DataFrame as parameter.
    """
    if type(df) != pd.core.frame.DataFrame:
        print('Query Cleaning nicht möglich - es wurde kein Pandas DataFrame übergeben.')
        print ('der Typ des Objekts ist ein {}'.format(type(df)) )
        return df
    
    prefixes = ['http://www.datacron-project.eu/datAcron#']
    prefixes.append('http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#')
    prefixes.append('http://www.w3.org/2001/XMLSchema#')
    prefixes.append('http://www.w3.org/2000/01/rdf-schema#')
    prefixes.append('java:datAcronTester.unipi.gr.sparql_functions.')
    prefixes.append('http://www.w3.org/1999/02/22-rdf-syntax-ns#')
    prefixes.append('http://www.openlinksw.com/schemas/virtrdf#')
    
    for item in prefixes:
        for column in df:
            df[column].replace(regex=True, inplace=True, to_replace=item, value='')
    return df

In [3]:
def regexify(df, column):
    """
    Creates a REGEX expression which matches the values of a pd.DataFrame column OR-wise.
    E.g. the pattern matches if the search string contains one or more elements of the column.
    """
    regex = '('
    for i in range (len(df)):
        regex = regex + df.iloc[i][column] + '|'
    
    regex = regex[:-3] + ')'
    return regex    

# Tutorial-Queries from "SPARQL_queries_example.pdf"

The following (long) section represents my work trying to replicate the given example queries.

### Pull all concepts

The first query returns all concepts, for which at least one assertion exists in the triple.

In [4]:
sparql_query = """
PREFIX datp: <http://datacron-project.eu#>
SELECT DISTINCT ?Concept 
WHERE {
    GRAPH ?g {[] a ?Concept}
} LIMIT 100
"""

df = query_data(sparql_query, )
df = query_value_cleaning(df)
df.head(5)


URLError: <urlopen error [Errno 110] Connection timed out>

### Pull all properties

PROBLEM

This query should pull all properties from the RDF store that belong to the named graph ?g, which is defined as http://localhost:8890/DAV. It should get all properties, for which at least one instance exists in this graph.
The empty braces mean that subject and object are not of interest here and may be omitted by the query processor.

Unfortunately, this query yields no results, neither in the ...107 nor in the ...109 data store. 
In both cases, the result set is empty.


In [194]:
sparql_query = """PREFIX datp: <http://datacron-project.eu#>
SELECT DISTINCT ?Property 
WHERE {
GRAPH ?g { [] ?Property []} 
FILTER(?g=<http://localhost:8890/DAV>)
}
"""

df = query_data(sparql_query)
#df = query_value_cleaning(df)
df.head(10)



,Property


### Get more information on something (own work)

I am using property paths here to drill through the ontology graph. This seems to work fine with different Classes of the ontology.

In [72]:
sparql_query = """
PREFIX : <http://www.datacron-project.eu/datAcron#>
PREFIX dul: <http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#>

SELECT ?s
WHERE {
  ?s rdfs:subClassOf* :Trajectory .
}
"""
 #?s rdf:type/rdfs:subClassOf* :SpatiotemporalRegion 

df = query_data(sparql_query)
df = query_value_cleaning(df)
df.head(10)



,s
0,Trajectory
1,SyntheticTrajectory
2,ActualTrajectory
3,IntentedTrajectory
4,RegulatedTrajectory
5,FM_FlightPlanTrajectory
6,OpenTrajectory
7,ClosedTrajectory


For a class or instance of interest, we can use the DESCRIBE syntax to get some more information.

In [73]:
sparql_query = """
PREFIX : <http://www.datacron-project.eu/datAcron#>
PREFIX dul: <http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#>

DESCRIBE ?s
WHERE {
  ?s rdfs:subClassOf* :Trajectory .
}
"""
 #?s rdf:type/rdfs:subClassOf* :SpatiotemporalRegion 

df = query_data(sparql_query)
df = query_value_cleaning(df)
df.head(10)



,s,p,o
0,RegulatedTrajectory,comment,A trajectory that has been regulated
1,FM_FlightPlanTrajectory,comment,"A sub-class of intented trajectory, which is r..."
2,hasPart,domain,Trajectory
3,ClosedTrajectory,subClassOf,ActualTrajectory
4,ClosedTrajectory,comment,A Trajectory where the destination is reached
5,Trajectory,comment,A trajectory consists of a sequence of tempor...
6,OpenTrajectory,type,http://www.w3.org/2002/07/owl#Class
7,departureOfTrajectory,domain,Trajectory
8,Trajectory,http://purl.org/dc/elements/1.1/creator,datAcron-project
9,Trajectory,type,http://www.w3.org/2002/07/owl#Class


### Pull sector configs, affected airspaces, time periods and capacities (Page 2)

This query pulls some sector configurations which were selected in the past. For these configurations, it lists 10 affected airspaces, their period of effectivity and the defined capacity.

In [195]:
sparql_query = """
PREFIX : <http://www.datacron-project.eu/datAcron#>
PREFIX dul: <http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#>

SELECT ?config ?airspace ?capacity ?start ?end
WHERE {
  ?config a :FM_Configuration ;
          :hasCapacity ?capacity ;
          :configurationOfAirspace ?airspace ;
          dul:hasConstituent/:TimeStart ?start ;
          dul:hasConstituent/:TimeEnd ?end.
}
ORDER BY ?start
LIMIT 10
"""

df = query_data(sparql_query)
df = query_value_cleaning(df)
df.to_csv('data/configs_and_affected_airspaces1.csv')
df.head(5)



,config,airspace,capacity,start,end
0,AirspaceConfiguration_LFMMXCTA_CF1_411,Airspace_LFMMXCTA_411,999,2016-03-31T00:00:00,2016-04-01T23:59:00
1,AirspaceConfiguration_LFMMXCTA_CF1_411,Airspace_LFMMXCTA_411,999,2016-03-31T00:00:00,2016-04-02T23:59:00
2,AirspaceConfiguration_LFMMXCTA_CF1_411,Airspace_LFMMXCTA_411,999,2016-03-31T00:00:00,2016-04-03T23:59:00
3,AirspaceConfiguration_LFMMXCTA_CF1_411,Airspace_LFMMXCTA_411,999,2016-03-31T00:00:00,2016-04-04T23:59:00
4,AirspaceConfiguration_LFMMXCTA_CF1_411,Airspace_LFMMXCTA_411,999,2016-03-31T00:00:00,2016-04-05T23:59:00


### Configurations and their sectors (own work)

If we want to know which configuration ever affected which sector, we can alter the above query: we omit the timestamp and select only DISTINCT values.

In [197]:
sparql_query = """
PREFIX : <http://www.datacron-project.eu/datAcron#>
PREFIX dul: <http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#>

SELECT DISTINCT * 
WHERE {
  ?config a :FM_Configuration ;
          :hasCapacity ?capacity ;
          :configurationOfAirspace ?airspace .
}
LIMIT 25000
"""

df = query_data(sparql_query)
df = query_value_cleaning(df)
%time df = df.sort_values('airspace')
df.to_csv('data/configs_and_affected_airspaces2.csv')
df.head(5)



CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 1.94 ms


,config,capacity,airspace
0,AirspaceConfiguration_BIRDCTA_CONF1_411,999,Airspace_BIRDCTA_411
1,AirspaceConfiguration_BIRDICTA_CNF1_411,999,Airspace_BIRDICTA_411
2,AirspaceConfiguration_BIRDTOCA_CONF1_411,999,Airspace_BIRDTOCA_411
3,AirspaceConfiguration_DAAACTA_CONF1_411,999,Airspace_DAAACTA_411
4,AirspaceConfiguration_DAAATCTA_CNF1_411,999,Airspace_DAAATCTA_411


RESULT: Technical result: to avoid timeouts, it is better to pull the data first and let Pandas do the sorting. If I include an ORDER BY clause, the query will timeout. Instead, I use ```pandas df.sort_values()``` method.

### Pull airspaces with capacity != 999 (Page 2)

The following query pulls only those configurations, where a capacity number != 999 was set, by using a FILTER operator.

In [200]:
sparql_query = """
PREFIX : <http://www.datacron-project.eu/datAcron#>
PREFIX dul: <http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#>

SELECT * 
WHERE {
   ?c a :FM_Configuration ;
      :hasCapacity ?capacity ;
      :configurationOfAirspace ?airspace ;
      dul:hasConstituent/:TimeStart ?start ;
      dul:hasConstituent/:TimeEnd ?end.
   FILTER(?capacity !='999')
}
"""

df = query_data(sparql_query)
df = query_value_cleaning(df)
df = df.sort_values('airspace')
df.to_csv('data/capacity_limited_configs.csv')
df.head(5)



,c,capacity,airspace,start,end
993,AirspaceConfiguration_EBBUCTA_CE2W2L_411,38,Airspace_EBBUCTA_411,2016-04-19T08:00:00,2016-04-19T06:59:00
973,AirspaceConfiguration_EBBUCTA_CE2W2L_411,38,Airspace_EBBUCTA_411,2016-04-13T08:00:00,2016-04-19T08:59:00
974,AirspaceConfiguration_EBBUCTA_CE2W2L_411,38,Airspace_EBBUCTA_411,2016-04-13T08:00:00,2016-04-19T08:59:00
975,AirspaceConfiguration_EBBUCTA_CE2W2L_411,38,Airspace_EBBUCTA_411,2016-04-13T08:00:00,2016-04-19T08:59:00
977,AirspaceConfiguration_EBBUCTA_CE2W2L_411,38,Airspace_EBBUCTA_411,2016-04-19T04:40:00,2016-04-13T08:59:00


### Configs and airspaces with limited capacity: only DISCTINCT values (own work)

I want to know if there is a direct relation between a configuration ID and the resulting airspace capacity. If this is the case, then the following query should only return one row per configuration ID.


In [202]:
sparql_query = """
PREFIX : <http://www.datacron-project.eu/datAcron#>
PREFIX dul: <http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#>

SELECT DISTINCT * 
WHERE {
   ?config a :FM_Configuration ;
           :hasCapacity ?capacity ;
           :configurationOfAirspace ?airspace .
   FILTER(?capacity !='999')
}
"""

df = query_data(sparql_query)
df = query_value_cleaning(df)
df.to_csv('data/capacity_limited_configs2.csv')
df.iloc[9:14]



,config,capacity,airspace
9,AirspaceConfiguration_EDWWCTAE_E6C_411,30,Airspace_EDWWCTAE_411
10,AirspaceConfiguration_EDWWCTAE_E7A_411,15,Airspace_EDWWCTAE_411
11,AirspaceConfiguration_EDWWCTAE_E7A_411,30,Airspace_EDWWCTAE_411
12,AirspaceConfiguration_EDWWCTAE_E8A_411,15,Airspace_EDWWCTAE_411
13,AirspaceConfiguration_EDWWCTAE_E8A_411,30,Airspace_EDWWCTAE_411


RESULT SEMANTICAL: as we can see with airspace configuration AirspaceConfiguration_EDWWCTAE_E7A_411 (EDWW is Bremen Radar), there are two different capacities for a single configuration. Therefore, the assumption that a direct dependency from configuration --> capacity exists, does not hold. 

It has to be noted though, that, looking at spanish sector configs, I could not find duplicate entries. TODO let Jose Manuel confirm that.



### Restricting the sector configuration query to a specific airspace (Page 3)

PROBLEM

The following query pulls all configurations that belong to the Airspace Airspace_LSAGCTA (Geneva Control Area). Unfortunately, it returns more than 10000 results, most of them are illogical, because the airspace is configured several times at the same time, which semantically makes no sense according to my understanding of the ATC environment.


In [74]:
sparql_query = """
PREFIX : <http://www.datacron-project.eu/datAcron#>
PREFIX dul: <http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#>

SELECT * 
WHERE {
  ?config a :FM_Configuration ;
          :hasCapacity ?capacity ;
          :configurationOfAirspace  :Airspace_LSAGCTA_411 ;
          dul:hasConstituent/:TimeStart ?start ;
          dul:hasConstituent/:TimeEnd ?end.
}
"""

df = query_data(sparql_query)
df = query_value_cleaning(df)
df = df.sort_values('start')
df.to_csv('data/configs_of_geneva.csv')
len(df)



10000

In [36]:
df.head(10)

,config,capacity,start,end
0,AirspaceConfiguration_LSAGCTA_I1A_411,40,2016-04-01T00:00:00,2016-04-01T03:54:00
565,AirspaceConfiguration_LSAGCTA_I1A_411,40,2016-04-01T00:00:00,2016-04-18T10:44:00
566,AirspaceConfiguration_LSAGCTA_I1A_411,40,2016-04-01T00:00:00,2016-04-18T10:49:00
567,AirspaceConfiguration_LSAGCTA_I1A_411,40,2016-04-01T00:00:00,2016-04-18T10:55:00
568,AirspaceConfiguration_LSAGCTA_I1A_411,40,2016-04-01T00:00:00,2016-04-18T10:59:00
569,AirspaceConfiguration_LSAGCTA_I1A_411,40,2016-04-01T00:00:00,2016-04-18T11:04:00
570,AirspaceConfiguration_LSAGCTA_I1A_411,40,2016-04-01T00:00:00,2016-04-18T14:51:00
571,AirspaceConfiguration_LSAGCTA_I1A_411,40,2016-04-01T00:00:00,2016-04-18T14:54:00
572,AirspaceConfiguration_LSAGCTA_I1A_411,40,2016-04-01T00:00:00,2016-04-18T15:09:00
573,AirspaceConfiguration_LSAGCTA_I1A_411,40,2016-04-01T00:00:00,2016-04-18T15:14:00


PROBLEM:

Instead of 2049 results as according to the tutorial, I get over 10000 configurations for the Geneva airspace?  And they all overlap in time?


NOTE
The syntactic subtlety: obviously, 'Airspace_LSAGCTA_411' is not the name but the ID of the Airspace in the triple store. Therefore, we do not need to search for a name and then combine the queries; it is sufficent to enter the Id directly with the ":" syntax.  


### Further inspect a specific airspace: drilling down to the airblocks (own work)

By traversing the ontology, we should be able to drill down to the airblocks and get their properties.
The query below shows us that our airspace of interest, Airspace_LSAGCTA_411 consist of three sub-airspaces: Airspace_LSAGE_411, Airspace_LSAGN_411, Airspace_LSAGS_411.

RESULT SEMANTICAL:

LSAGN is Geneva North, LSAGE is Geneva East and LSAGS is Geneva South. The appendix _411 refers to the AIRAC cycle of this airspace. Reminder: the shape of an airspace may change at the change of an airac cycle. Therefore, the design decision to add the appendix _411 directly to the name of the airspace guarantees that there should be only one geometry per airspace ID.

In [193]:
sparql_query = """
PREFIX : <http://www.datacron-project.eu/datAcron#>
PREFIX dul: <http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#>

SELECT ?p ?o
WHERE {
  :Airspace_LSAGCTA_411 ?p ?o .
}
LIMIT 2000
"""

df = query_data(sparql_query)
df = query_value_cleaning(df)
df.to_csv('data/airspace_properties.csv')
df.head(10)

,p,o
0,type,FM_Airspace
1,hasPart,Airspace_LSAGE_411
2,hasPart,Airspace_LSAGN_411
3,hasPart,Airspace_LSAGS_411


Lets inspect these sub-airspaces, and look what they are made of. I somehow hope to reconstruct the shape of the airspaces according to their official AIP (here an example of Geneva East Part 1, according SWISSCONTROL):



<img src="images/geneva_east.png"></img>

In [194]:
sparql_query = """
PREFIX : <http://www.datacron-project.eu/datAcron#>
PREFIX dul: <http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#>

SELECT  ?s ?p ?o
WHERE { 
  VALUES ?s { :Airspace_LSAGE_411 :Airspace_LSAGN_411 :Airspace_LSAGS_411}
  VALUES ?p { dul:hasPart}
  ?s dul:hasPart ?o. 
}

LIMIT 2000
"""

# to check spanish airspace blocks, give values :Airspace_LECMASU_411 
#              :Airspace_LECMBLL_411 :Airspace_LECMBLU_411 
#              :Airspace_LECMCJL_411 :Airspace_LECMCJU_411 :Airspace_LECMDGL_411
#              :Airspace_LECMDGU_411 :Airspace_LECMPAL_411

blocks = query_data(sparql_query)
blocks = query_value_cleaning(blocks)
blocks.to_csv('data/airspace_inspection.csv')

blocks.head(10)

,s,p,o
0,Airspace_LSAGE_411,hasPart,Airblock_LSAGE_LSAGE_301LS_75_245_75_245
1,Airspace_LSAGE_411,hasPart,Airblock_LSAGE_LSAGE_311LS_75_245_75_245
2,Airspace_LSAGE_411,hasPart,Airblock_LSAGE_LSAGE_503LS_155_245_155_245
3,Airspace_LSAGE_411,hasPart,Airblock_LSAGE_LSAGE_513LS_155_245_155_245
4,Airspace_LSAGE_411,hasPart,Airblock_LSAGE_LSAGE_716LF_155_245_155_245
5,Airspace_LSAGE_411,hasPart,Airblock_LSAGE_LSAGE_750LS_75_245_75_245
6,Airspace_LSAGE_411,hasPart,Airblock_LSAGE_LSAGE_778LS_75_245_75_245
7,Airspace_LSAGE_411,hasPart,Airblock_LSAGE_LSAGE_802LS_130_245_130_245
8,Airspace_LSAGE_411,hasPart,Airblock_LSAGE_LSAGE_804LS_75_245_75_24
9,Airspace_LSAGN_411,hasPart,Airblock_LSAGN_LSAGN_080LF_155_195_155_195


RESULT SEMANTICAL

The sub-Airspaces consist of airblocks. It should be possible to inspect these airblocks in more detail. Lets first check which properties are available for the Class FM_Airblock.

In [179]:
sparql_query = """ 
PREFIX : <http://www.datacron-project.eu/datAcron#>
PREFIX dul: <http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#>

SELECT DISTINCT ?p WHERE { ?s rdf:type :FM_Airblock ;   ?p ?o .} LIMIT 100 """

df = query_data(sparql_query)
query_value_cleaning(df)
df.head(10)

,p
0,type
1,hasLowerLevel
2,hasUpperLevel
3,hasGeometry


RESULT SEMANTICAL

OK, so the airblocks should, as described in ontology and tutorial, be comprised out of :hasLowerLevel, :hasUpperLevel and :hasGeometry. Lets try to get these properties for the airblocks of the three Geneva airspaces LSAGE LSAGN and LSAGS. I will try a single airblock first:

In [180]:
sparql_query = """ 
PREFIX : <http://www.datacron-project.eu/datAcron#>
PREFIX dul: <http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#>

SELECT ?ll ?ul ?geom 
WHERE {
      :Airblock_LSAGE_LSAGE_301LS_75_245_75_245   :hasLowerLevel ?ll ;
                                                  :hasUpperLevel ?ul ;
                                                  :hasGeometry ?geom .                                   
}
"""
df = query_data(sparql_query)
query_value_cleaning(df)
#df.to_csv("airspaceconfigs.csv")
df.head(5)

,ll,ul,geom


PROBLEM

No luck here; the properties seem to be empty. Maybe a missing value or some problem with the query? Lets check the complete set of airblocks by creating a list and passing it to SPARQL via the VALUES snytax.

In [181]:
# Construct the list of airblocks
str = ''
for i in range(len(blocks)):
    if blocks.iloc[i]['p'] == 'hasPart':
        str = str +' :' + blocks.iloc[i]['o']
str = str[1:]
        
# check the list
print(str[:300] + '...')

:Airblock_LSAGE_LSAGE_301LS_75_245_75_245 :Airblock_LSAGE_LSAGE_311LS_75_245_75_245 :Airblock_LSAGE_LSAGE_503LS_155_245_155_245 :Airblock_LSAGE_LSAGE_513LS_155_245_155_245 :Airblock_LSAGE_LSAGE_716LF_155_245_155_245 :Airblock_LSAGE_LSAGE_750LS_75_245_75_245 :Airblock_LSAGE_LSAGE_778LS_75_245_75_245 ...


In [182]:
#Inject the list of airblocks into the sparql query and get all properties of all airblocks
sparql_query2 = """
PREFIX : <http://www.datacron-project.eu/datAcron#>
PREFIX dul: <http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#>

SELECT DISTINCT ?s ?ll ?ul ?geom 
WHERE {
      VALUES ?s {""" + str + """}
      ?s :hasLowerLevel ?ll ;
         :hasUpperLevel ?ul ;
         :hasGeometry ?geom .                                   
}
"""
df2 = query_data(sparql_query2)
df2 = query_value_cleaning(df2)
%time df2 = df2.sort_values('s')

df2

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 625 µs


,s,ll,ul,geom


PROBLEM

Unfortunately, the query does not return more details on any airblock ID that I enter into the parameters. Normally, I would have excpected to get the 3D coordinates of the airblocks. This point prevents me from proceeding with my regulations analysis.

NOTE

The same traversal down to the airblocks was also done with Madrid airspace, with the same (zero) results. It seems that there is something wrong. 

But according to the following query, I can get a comprehensive list of airblocks! Some REGEX was applied to get the Geneva airblocks out of otherwise 10.000+ results.

In [195]:
sparql_query = """ 
PREFIX : <http://www.datacron-project.eu/datAcron#>
PREFIX dul: <http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#>

SELECT DISTINCT ?block ?ll ?ul ?geom 
WHERE {
      ?block :hasLowerLevel ?ll ;
             :hasUpperLevel ?ul ;
             :hasGeometry ?geom .
      
      FILTER regex(?block, 'Airblock_LSAGE', "i")
} 
"""

#FILTER regex(?y, "YOUR_REGEX", "i")
#?block bif:contains "'Airblock_LEMD*'"
#FILTER regex(?block, '(.*Airblock_LSAGN2_507LS_195_245.* | .*Airblock_LSAGN2_700LF_195_245.* ) ', "i")
#FILTER regex(?block, 'LSAGN', "i")

df = query_data(sparql_query)
query_value_cleaning(df)
df.to_csv("data/blocks_with_geometry.csv")
df.tail(12)

,block,ll,ul,geom
159,Airblock_LSAGEUROPE_779LI_355_999,10820.4,30449.52,http://83.212.239.107/geometries/airblocks/LSA...
160,Airblock_LSAGEUROPE_902LF_355_999,10820.4,30449.52,http://83.212.239.107/geometries/airblocks/LSA...
161,Airblock_LSAGEUROPE_948LI_355_999,10820.4,30449.52,http://83.212.239.107/geometries/airblocks/LSA...
162,Airblock_LSAGE_301LS_75_245,2286.0,7467.6,http://83.212.239.107/geometries/airblocks/LSA...
163,Airblock_LSAGE_311LS_75_245,2286.0,7467.6,http://83.212.239.107/geometries/airblocks/LSA...
164,Airblock_LSAGE_503LS_155_245,4724.400000000001,7467.6,http://83.212.239.107/geometries/airblocks/LSA...
165,Airblock_LSAGE_513LS_155_245,4724.400000000001,7467.6,http://83.212.239.107/geometries/airblocks/LSA...
166,Airblock_LSAGE_716LF_155_245,4724.400000000001,7467.6,http://83.212.239.107/geometries/airblocks/LSA...
167,Airblock_LSAGE_750LS_75_245,2286.0,7467.6,http://83.212.239.107/geometries/airblocks/LSA...
168,Airblock_LSAGE_778LS_75_245,2286.0,7467.6,http://83.212.239.107/geometries/airblocks/LSA...


POSSIBLE SOLUTION TO PROBLEM

As we can see here, the airblocks of LSAGE seem to have different names in this query than in the other query!
In the other query (above) retrieving the airblocks of airspace Airspace_LSAGE_411 resulted in names like this:
 
 - Airblock_LSAGE_LSAGE_301LS_75_245_75_245
 - Airblock_LSAGE_LSAGE_311LS_75_245_75_245
 - Airblock_LSAGE_LSAGE_503LS_155_245_155_245
 - Airblock_LSAGE_LSAGE_513LS_155_245_155_245
  
whereas via this query, we get totally different names! We get names like this:

 - Airblock_LSAGE_301LS_75_245
 - Airblock_LSAGE_311LS_75_245
 - Airblock_LSAGE_503LS_155_245
 - Airblock_LSAGE_513LS_155_245
  

The last numbers obviously indicate the flight level range of the airblocks. But somehow, in the first query, they seem to be annotated twice to the Airblock ID. This should clearly not be the case. Is there something wrong in the database?
  


### Get frequency of configurations applied (Page 3)

This query returns the number of configurations that have been applied to the sector.

TODO / PROBLEM

I cannot agree with the tutorial which states that this query returns the FREQUENCY of configurations applied; because the notion of FREQUENCY implies that ther should be a time dimension in the denominator ( / s), but there is no time dimension in the following query. 

Instead, I think that this query just counts the total number of configurations applied where, due to some other decision, the flow manager applied a capacity != 999.

In [196]:
sparql_query = """ 
PREFIX : <http://www.datacron-project.eu/datAcron#>
PREFIX dul: <http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#>

SELECT (COUNT(?x) as ?count) ?a
WHERE {
    ?x :configurationOfAirspace ?a ;
       :hasCapacity ?capacity .
    FILTER(?capacity != "999")
} 
GROUP BY ?a
ORDER BY DESC(?count)
"""
df = query_data(sparql_query)
query_value_cleaning(df)
#df.to_csv("airspaceconfigs.csv")
df.head(10)

,count,a
0,60,Airspace_LEMDTMA_411
1,60,Airspace_LFMMCTAW_411
2,50,Airspace_LSAZUTA_411
3,40,Airspace_LECMCTAN_411
4,28,Airspace_LYBACTA_411
5,26,Airspace_LECMCTAS_411
6,26,Airspace_LOVVCTA_411
7,20,Airspace_LEBLTMA_411
8,20,Airspace_LECPCTA_411
9,18,Airspace_LECSCTA_411


### Get all configurations in a specific time frame (Page 3)

The following query gets all configurations on 28th of April, 2016 etween 00:00 and 05:00 UTC. 

PROBLEM

The query according tutorial is malformed in two ways: 
1. The Airspace ID is Airspace_LSAGCTA_411, not Airspace_LSAGCTA (thanks to Christos for pointing this out) 
2. The result set for the given time frame is empty. If I alter the time frame to be from 27th of April, I get some (max 10.000) results. But 10.000 configurations for a 29 hour time period? This seems odd to me.
3. If I inspect the results (for example, take row 0), then I get configurations that started at the 27th of April, but ended at the 1st of April .... which is at least mind-bending.

ASSUMPTION:
With the query syntax like below, we are still OR-ing the restrictions of the results. Therefore, we get all configs that started after ?start or ended before ?end. Which are basically, all configurations.


In [197]:
sparql_query = """ 
PREFIX : <http://www.datacron-project.eu/datAcron#>
PREFIX dul: <http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT ?config ?capacity
WHERE {
    ?config a :FM_Configuration;
            :hasCapacity ?capacity ;
            :configurationOfAirspace :Airspace_LSAGCTA_411 ;
            dul:hasConstituent/:TimeStart ?start ;
            dul:hasConstituent/:TimeEnd ?end .
    FILTER( str(?start)>"2016-04-28T00:00:00" && str(?end)<"2016-04-28T05:00:00" )
}

"""
df = query_data(sparql_query)
query_value_cleaning(df)
df.to_csv("data/airspaceconfigs_0000_0500.csv")
df.head(5)

,config,capacity


So, zero entries? This does not seem reasonable. So lets say that the string comparison of times gives odd results. Let's try this again with the xsd namespace like in the tutorial. 

In [198]:
sparql_query = """ 
PREFIX : <http://www.datacron-project.eu/datAcron#>
PREFIX dul: <http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT ?config ?capacity
WHERE {
    ?config a :FM_Configuration;
            :hasCapacity ?capacity ;
            :configurationOfAirspace :Airspace_LSAGCTA_411 ;
            dul:hasConstituent/:TimeStart ?start ;
            dul:hasConstituent/:TimeEnd ?end .
    FILTER( ?start > xsd:dateTime("2016-04-28 00:00:00") && ?end < xsd:dateTime("2016-04-28 05:00:00") )
}

"""
df = query_data(sparql_query)
query_value_cleaning(df)
df.to_csv("data/airspaceconfigs_0000_0500.csv")
df.head(5)

,config,capacity


Same odd results. If I now alter the time frame to 27th of April, I get the following, illogical results: more than 10.000 configurations applied within 29 hours for a single airspace .... seems odd to me.

In [192]:
sparql_query = """ 
PREFIX : <http://www.datacron-project.eu/datAcron#>
PREFIX dul: <http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT ?config ?capacity ?start ?end
WHERE {
    ?config a :FM_Configuration;
            :hasCapacity ?capacity ;
            :configurationOfAirspace :Airspace_LSAGCTA_411 ;
            dul:hasConstituent/:TimeStart ?start ;
            dul:hasConstituent/:TimeEnd ?end .
    FILTER( ?start > xsd:dateTime("2016-04-27 00:00:00") && ?end < xsd:dateTime("2016-04-28 05:00:00") )
}

"""
df = query_data(sparql_query)
query_value_cleaning(df)
df.to_csv("data/airspaceconfigs_0000_0500.csv")
len(df)

10000

## Trajectories Inspection (Page 3 bottom)

The following query pulls 100 trajectories out of the store, directly with their coordinates.

NOTE

Be aware that the direct availability of the coordinates is a convenience function created by the RDF maintainers. It prevents us from needing to traverse into every single semantic node of a tracjectory to get the coordinates and allows for easy plotting.

PROBLEM

Unfortunately, this convenience function only gives us 2D coordinates without altitude or time information. But what we need in order to cross trajectories and airblocks is the 4D coordinates including time and altitude. The chosen data format, well known text (WKT) would able to handle 4D coordinates, because z coordinate is available for altitude rep and m coordinate for linear data (time passed since epoch could be considered as a linear distance of a point from the epoch).


In [199]:
sparql_query = """ 
PREFIX : <http://www.datacron-project.eu/datAcron#>
PREFIX dul: <http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#>

SELECT DISTINCT * WHERE {
  ?flight_plan_ID a :FM_RTFM ;
                    :departureAirport ?dep ;
                    :destinationAirport ?dest ;
                    :reportsTrajectory ?trajectory_ID .
  ?trajectory_ID dul:hasConstituent/:hasWKT ?geom

} LIMIT 100

"""

#to query another endpoint, change the URL for the service and the query
df = query_data(sparql_query)
query_value_cleaning(df)
df.to_csv("data/trajectories.csv")
df.head(5)

,flight_plan_ID,dep,dest,trajectory_ID,geom
0,flight_plan_AA51114077,Place_MontpellierMediterranee_Airport,Place_BastiaPoretta_Airport,traj_AA51114077_20160401003000,"LINESTRING(3.961389 43.583333, 3.968056 43.580..."
1,flight_plan_AA51114336,Place_Bristol___Lulsgate,Place_TenerifeSur_ReinaSofia_Airport,traj_AA51114336_20160331154500,"LINESTRING(-2.719167 51.382778, -2.736111 51.3..."
2,flight_plan_AA51114524,Place_Valencia_Manises_Airport,Place_Bucuresti_HenriCoanda_Airport,traj_AA51114524_20160331183000,"LINESTRING(-0.481667 39.489444, -0.492222 39.4..."
3,flight_plan_AA51115368,Place_Nantes_Atlantique_Airport,Place_London___Gatwick_Airport,traj_AA51115368_20160331155000,"LINESTRING(-1.607778 47.156944, -1.601389 47.1..."
4,flight_plan_AA51121974,Place_QuimperPluguffan_Airport,Place_ParisOrly_Airport,traj_AA51121974_20160401044000,"LINESTRING(-4.167778 47.975, -4.143056 47.9677..."


#### Visualizing the trajectories 

We can visualize these trajectories in 2D using gemoet [https://pypi.python.org/pypi/geomet/0.1.0] for  WKT-->GeoJson transformation and folium [https://github.com/python-visualization/folium], which is a python wrapper for Leaflet.js interactive maps.

In [200]:
from geomet import wkt
import json

#Convert WKT column into a gejson column
df["geojson"] = df["geom"].apply(lambda x: json.dumps(wkt.loads(x)))
df = df.drop('geom', 1)
df.head(5)

,flight_plan_ID,dep,dest,trajectory_ID,geojson
0,flight_plan_AA51114077,Place_MontpellierMediterranee_Airport,Place_BastiaPoretta_Airport,traj_AA51114077_20160401003000,"{""type"": ""LineString"", ""coordinates"": [[3.9613..."
1,flight_plan_AA51114336,Place_Bristol___Lulsgate,Place_TenerifeSur_ReinaSofia_Airport,traj_AA51114336_20160331154500,"{""type"": ""LineString"", ""coordinates"": [[-2.719..."
2,flight_plan_AA51114524,Place_Valencia_Manises_Airport,Place_Bucuresti_HenriCoanda_Airport,traj_AA51114524_20160331183000,"{""type"": ""LineString"", ""coordinates"": [[-0.481..."
3,flight_plan_AA51115368,Place_Nantes_Atlantique_Airport,Place_London___Gatwick_Airport,traj_AA51115368_20160331155000,"{""type"": ""LineString"", ""coordinates"": [[-1.607..."
4,flight_plan_AA51121974,Place_QuimperPluguffan_Airport,Place_ParisOrly_Airport,traj_AA51121974_20160401044000,"{""type"": ""LineString"", ""coordinates"": [[-4.167..."


In [201]:
#Lets inscpect how the geojson between Bristol and Tenerife looks like. 
df.iloc[1]['geojson']

'{"type": "LineString", "coordinates": [[-2.719167, 51.382778], [-2.736111, 51.382778], [-2.787222, 51.382222], [-2.838333, 51.381944], [-2.866389, 51.366111], [-2.894722, 51.350278], [-2.998056, 51.2925], [-3.359722, 51.178611], [-3.362778, 51.161111], [-3.371944, 51.108889], [-3.384167, 51.039167], [-3.388611, 51.013056], [-3.398056, 50.957778], [-3.416944, 50.8475], [-3.423889, 50.809167], [-3.431944, 50.761667], [-3.448333, 50.666667], [-3.4525, 50.641389], [-3.463889, 50.574167], [-3.473333, 50.518611], [-3.493611, 50.398611], [-3.495556, 50.362778], [-3.505278, 50.184167], [-3.517222, 49.960556], [-3.529444, 49.737222], [-3.558333, 49.658611], [-3.638611, 49.44], [-3.728333, 49.195278], [-3.783056, 49.046944], [-3.808611, 48.976944], [-4.061667, 48.301944], [-4.184722, 47.957222], [-4.3125, 47.829444], [-4.524722, 47.591389], [-4.990556, 47.079167], [-5.408056, 46.612778], [-6.919722, 44.835278], [-7.012778, 44.698056], [-7.143889, 44.504444], [-8.926667, 41.872778], [-9.016111, 

In [202]:
import folium


map1 = folium.Map(location=[40,10], zoom_start=4, control_scale=True, prefer_canvas=True)
for index, row in df.head(5).iterrows():
    c = folium.GeoJson(row['geojson'], name = (row['dep']+ row['dest']),overlay=True, 
                       style_function = lambda feature: {'fillColor': '#ffaf00','color': 'blue', 'weight': 2.5,'dashArray': '5, 5'},
                       highlight_function = lambda feature: {'fillColor': '#ffaf00','color': 'green', 'weight': 5,'dashArray': '5, 5'})
    c.add_child(folium.Popup(row['trajectory_ID'] + '\n' + row['dep'] +'\n' + row['dest']))
    c.add_to(map1)
folium.LayerControl().add_to(map1)
map1.save(outfile='maps/map1.html')

map1    

### Further inspecting trajectories (Page 4 bottom)

According to the tutorial, the following query should give us all semantic nodes and their linked information.

PROBLEM

see below.

In [208]:
sparql_query = """ 
PREFIX : <http://www.datacron-project.eu/datAcron#>
PREFIX dul: <http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#>

SELECT ?node ?time ?position 
WHERE {
   :traj_AA51114077_20160401003000 :hasPart ?node .
   ?node dul:hasConstituent/:TimeStart ?time ;
         dul:hasConstituent/:hasWKT ?position ; 
         ?p ?o
} ORDER BY ?time

"""

#traj_AA51114077_20160401003000
#traj_AA51114077_20160401003000_m1  <---- according tutorial, we should use, but gives 0 results

tutorialtraj = query_data(sparql_query)
query_value_cleaning(tutorialtraj)
tutorialtraj.to_csv("data/node.csv")
len(tutorialtraj)

3004

In [209]:
tutorialtraj.head(20)

,node,time,position
0,n_AA51114077_20160401003000_1,2016-04-01T03:52:30,POINT(3.961389 43.583333)
1,n_AA51114077_20160401003000_1,2016-04-01T03:52:30,POINT(3.961389 43.583333)
2,n_AA51114077_20160401003000_1,2016-04-01T03:52:30,POINT(3.961389 43.583333)
3,n_AA51114077_20160401003000_1,2016-04-01T03:52:30,POINT(3.961389 43.583333)
4,n_AA51114077_20160401003000_1,2016-04-01T03:52:30,POINT(3.961389 43.583333)
5,n_AA51114077_20160401003000_1,2016-04-01T03:52:30,POINT(3.961389 43.583333)
6,n_AA51114077_20160401003000_2,2016-04-01T03:52:49,POINT(8.626944 43.118333)
7,n_AA51114077_20160401003000_2,2016-04-01T03:52:49,POINT(8.626944 43.118333)
8,n_AA51114077_20160401003000_2,2016-04-01T03:52:49,POINT(9.506944 42.661944)
9,n_AA51114077_20160401003000_2,2016-04-01T03:52:49,POINT(9.506944 42.661944)


PROBLEM 

This representation does not seem logical. How is it possible that one and the same node can have multiple timestamps and multiple positions? Is it even realistic that a flight has 3004 informations on semantic nodes? I do not think so. In fact, I do think that there is something wrong with how the data is queried or how it is stored in the data store. 



May be something is wrong with this trajectory,. Let me check another:

(Bristol-Teneriffa traj_AA51114336_20160331154500)
                   
Lets see what properties for a single node are available:

In [211]:
sparql_query = """ 
PREFIX : <http://www.datacron-project.eu/datAcron#>
PREFIX dul: <http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#>

SELECT DISTINCT ?p ?o 
WHERE {
    :n_AA51114336_20160331154500_1  ?p ?o
}

"""
node = query_data(sparql_query)
query_value_cleaning(node)
node.to_csv("data/node.csv")
node

,p,o
0,hasConstituent,t_1459445125
1,hasConstituent,geom_EGGD
2,precedes,n_AA51114336_20160331154500_2


Lets get all the data for the trajectory between Bristol and Tenerife (like the query from page 4). We would excpect that the reulsting nodes exaclty comprise the trajectory that was plotted above. 

In [212]:
sparql_query = """ 
PREFIX : <http://www.datacron-project.eu/datAcron#>
PREFIX dul: <http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#>

SELECT *
WHERE {
    :traj_AA51114336_20160331154500 :hasPart ?node .
    ?node dul:hasConstituent/:TimeStart ?time ;
          dul:hasConstituent/:hasWKT ?position ;
          ?p ?o .      
}

"""

singletraj = query_data(sparql_query)
query_value_cleaning(singletraj)
singletraj.to_csv("data/singletrajectory.csv")

len(singletraj)

10000

PROBLEM

The trajecotry consists of 10.000 nodes? This does not seem resonable to me. I will now try to get some better syntax into the query.

In [213]:
sparql_query = """ 
PREFIX : <http://www.datacron-project.eu/datAcron#>
PREFIX dul: <http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#>

SELECT *
WHERE {
    :traj_AA51114336_20160331154500 :hasPart ?node .
    ?node dul:hasConstituent/:TimeStart ?time .
    ?node dul:hasConstituent/:hasWKT ?position .      
}

"""

singletraj = query_data(sparql_query)
query_value_cleaning(singletraj)
singletraj.to_csv("data/singletrajectory.csv")

len(singletraj)

3504

Now, the resultset is "only" 3504 nodes. Lets clean the rows a little bit and get only distinct node IDs:

In [214]:
singletraj = singletraj.drop_duplicates(subset = 'node', keep = 'first')

singletraj = singletraj.sort_values('node')
singletraj.to_csv("data/singletrajectory_drop_duplicates.csv")
print(len(singletraj))
singletraj.head(10)

72


,node,time,position
3476,n_AA51114336_20160331154500_1,2016-03-31T17:25:25,POINT(-2.719167 51.382778)
320,n_AA51114336_20160331154500_10,2016-03-31T17:32:59,POINT(-16.604167 28.033056)
400,n_AA51114336_20160331154500_11,2016-03-31T17:33:42,POINT(-16.604167 28.033056)
3448,n_AA51114336_20160331154500_12,2016-03-31T17:33:57,POINT(-3.388611 51.013056)
480,n_AA51114336_20160331154500_13,2016-03-31T17:34:28,POINT(-16.604167 28.033056)
3442,n_AA51114336_20160331154500_14,2016-03-31T17:35:26,POINT(-3.416944 50.8475)
3454,n_AA51114336_20160331154500_15,2016-03-31T17:35:50,POINT(-3.423889 50.809167)
560,n_AA51114336_20160331154500_16,2016-03-31T17:36:14,POINT(-16.604167 28.033056)
3452,n_AA51114336_20160331154500_17,2016-03-31T17:37:00,POINT(-3.448333 50.666667)
2720,n_AA51114336_20160331154500_18,2016-03-31T17:37:14,POINT(-16.604167 28.033056)


PROBLEM

Possible BUG? Lets inspect this result. If we check out the third line, we see a node at 17:33:42 at some coordinates -16°E, 28°N, which is close to Tenerife. At the fourth row, we see a node at 17:33:57, so only 15 seconds later. Magically, the airplace is now at position -3°E, 51°N, which is around Bristol. So the airplane did not only travel backwards, it also did this incredibly fast!   I suppose that there must be a problem with the data or the way I query this data.

See further below for a map with the malformed trajectory.



# UNDER CONSTRUCTION  





everything below is "under construction"

### Reconstructing trajectories from nodes (own work)


The final goal is to recosntruct the rajectories from the nodes, in order to have "enriched" trajectories with time, altitude and weather information. For this, we try to get all positions and times into one GeoJSON file which can then be passed to the folium map. According to their tutorial, the format of the GeoJSON is as follows:

```
{
"type": "Feature",
"geometry": {"type": "LineString", 
             "coordinates": [[lon, 25] for
                                lon in np.linspace(-150, 150, 25)] }, 
"properties": {"times": [1435708800000+i*86400000 for i in np.linspace(0, 25, 25)] }
}
```
So we need to create a dict which is formatted accordingly. The functions defined in the following box will help me with this task.

In [227]:
def wkt_to_geojson(dataframe, wkt_column_name, geojson_column_name):
    """
    Gets a pandas.DataFrame and converts a WKT column into a GeoJSON column.
    The GeoJSON column then contains a dict.
    
    Keyword arguments:
    dataframe -- the pandas.DataFrame which should be altered
    wkt_column_name -- the name of the column containing the well-known-text
    geojson_column_name -- how the GeoJSON column shall be named
    
    """
    if type(dataframe) != pd.core.frame.DataFrame:
        raise TypeError('The parameter dataframe must be a Pandas DataFrame.')
    if not wkt_column_name in dataframe.columns:
        raise ValueError('No column with the specified name found in the DataFrame')
    
    dataframe[geojson_column_name] = dataframe[wkt_column_name].apply(lambda x: wkt.loads(x))
    return dataframe


def inject_time(geojson, time):
    """
    Injects Time dimension into geoJSON coordinates. Expects  a dict in geojson POINT format.
    """
    #geojson['coordinates'] = [geojson['coordinates'][0], geojson['coordinates'][1], time]
    geojson['properties'] = {'times' : [time]}
    return geojson

def extract_coordinates(geojson):
    """
    Returns only the coordinates from a geoJSON POINT object
    """
    return geojson['coordinates']





In [226]:
#convert position to geoJson
singletraj = wkt_to_geojson(singletraj, 'position', 'geojson')
#pull only coordinates out of the GeoJSON column
singletraj['coord_only'] =  singletraj['geojson'].apply(lambda x: extract_coordinates(x))
singletraj = singletraj.drop('geojson', 1)
singletraj = singletraj.drop('position', 1)
singletraj.head(5)

NameError: name 'dataframe' is not defined

In [219]:
#convert time string into milliseconds since epoch. After this, we can sort the resulting DataFrame by time.
from datetime import datetime

singletraj['time'] = singletraj['time'].apply(lambda x: datetime.strptime(x, '%Y-%m-%dT%H:%M:%S').timestamp())
singletraj = singletraj.sort_values('time')

singletraj.to_csv('data/singletraj_timestamped.csv')
singletraj.head(10)

,node,time,coord_only
3476,n_AA51114336_20160331154500_1,1.459438e+09,"[-2.719167, 51.382778]"
0,n_AA51114336_20160331154500_2,1.459438e+09,"[-16.604167, 28.033056]"
80,n_AA51114336_20160331154500_3,1.459438e+09,"[-16.604167, 28.033056]"
3456,n_AA51114336_20160331154500_4,1.459438e+09,"[-2.838333, 51.381944]"
160,n_AA51114336_20160331154500_5,1.459438e+09,"[-16.604167, 28.033056]"
240,n_AA51114336_20160331154500_6,1.459438e+09,"[-16.604167, 28.033056]"
3502,n_AA51114336_20160331154500_7,1.459438e+09,"[-2.998056, 51.2925]"
3446,n_AA51114336_20160331154500_8,1.459438e+09,"[-3.359722, 51.178611]"
1680,n_AA51114336_20160331154500_9,1.459438e+09,"[-16.604167, 28.033056]"
320,n_AA51114336_20160331154500_10,1.459438e+09,"[-16.604167, 28.033056]"


In [220]:
singletraj.head(5)

,node,time,coord_only
3476,n_AA51114336_20160331154500_1,1.459438e+09,"[-2.719167, 51.382778]"
0,n_AA51114336_20160331154500_2,1.459438e+09,"[-16.604167, 28.033056]"
80,n_AA51114336_20160331154500_3,1.459438e+09,"[-16.604167, 28.033056]"
3456,n_AA51114336_20160331154500_4,1.459438e+09,"[-2.838333, 51.381944]"
160,n_AA51114336_20160331154500_5,1.459438e+09,"[-16.604167, 28.033056]"


Now we have the times and the coordinates in a dict-friendly way. We can now iterate through the rows and create our custom GeoJSON object, that we can then pass to the folium map. 

In [228]:
#TODO use this function instead

def geojsonify(dataframe, time_column, coord_column):
    res = {}
    res['type'] = 'LineString'
    res['coordinates'] = []
    res['properties'] = {'times': []}

    for x in range(len(dataframe)):
        res['coordinates'].append(dataframe.iloc[x][coord_column])
        res['properties']['times'].append(dataframe.iloc[x][time_column])
    res2 = {}
    res2['type']  = 'Feature'
    res2['geometry'] = res
    return res2 

In [221]:
ng = {}
ng['type'] = 'LineString'
ng['coordinates'] = []
ng['properties'] = {'times': []}

for x in range(len(singletraj)):
    ng['coordinates'].append(singletraj.iloc[x]['coord_only'])
    ng['properties']['times'].append(singletraj.iloc[x]['time'])

ng2 = {}
ng2['type']  = 'Feature'
ng2['geometry'] = ng

ng2

{'geometry': {'coordinates': [[-2.719167, 51.382778],
   [-16.604167, 28.033056],
   [-16.604167, 28.033056],
   [-2.838333, 51.381944],
   [-16.604167, 28.033056],
   [-16.604167, 28.033056],
   [-2.998056, 51.2925],
   [-3.359722, 51.178611],
   [-16.604167, 28.033056],
   [-16.604167, 28.033056],
   [-16.604167, 28.033056],
   [-3.388611, 51.013056],
   [-16.604167, 28.033056],
   [-3.416944, 50.8475],
   [-3.423889, 50.809167],
   [-16.604167, 28.033056],
   [-3.448333, 50.666667],
   [-16.604167, 28.033056],
   [-3.463889, 50.574167],
   [-16.604167, 28.033056],
   [-3.493611, 50.398611],
   [-16.604167, 28.033056],
   [-16.604167, 28.033056],
   [-16.604167, 28.033056],
   [-3.529444, 49.737222],
   [-16.604167, 28.033056],
   [-16.604167, 28.033056],
   [-16.604167, 28.033056],
   [-16.604167, 28.033056],
   [-3.808611, 48.976944],
   [-4.061667, 48.301944],
   [-4.184722, 47.957222],
   [-4.3125, 47.829444],
   [-4.524722, 47.591389],
   [-4.990556, 47.079167],
   [-5.408056, 4

Lets show these coordinates as a trajectory in a map.

In [222]:
from folium.plugins import TimestampedGeoJson

map2 = folium.Map(location=[40,10], zoom_start=4, control_scale=True, prefer_canvas=True)
#tgj = TimestampedGeoJson(data = ng2, period = 'PT1M')
#map2.add_child(tgj, name='sometimestampedgeojson')


c = folium.GeoJson(ng2, name = 'Traj with time',overlay=True, 
                       style_function = lambda feature: {'fillColor': '#ffaf00','color': 'blue', 'weight': 2.5,'dashArray': '5, 5'},
                       highlight_function = lambda feature: {'fillColor': '#ffaf00','color': 'green', 'weight': 5,'dashArray': '5, 5'})
c.add_child(folium.Popup('some popup'))
c.add_to(map2)
folium.LayerControl().add_to(map2)
map2.save(outfile='maps/map2.html')

map2  

In [223]:



#Convert WKT column into a gejson column
df["geojson"] = df["geom"].apply(lambda x: json.dumps(wkt.loads(x)))
df = df.drop('geom', 1)
df.head(3)

po = json.loads(singletraj.iloc[0]['geojson'])

print(po)
po['coordinates'] = [po['coordinates'][0], po['coordinates'][1], singletraj.iloc[0]['time'] ]
print(po)
type(po)

KeyError: 'geom'

In [311]:

sparql_query = """ 
PREFIX : <http://www.datacron-project.eu/datAcron#>
PREFIX dul: <http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#>

SELECT * WHERE {
    :n_AA51114077_20160401003000_1 ?p ?o .
}
"""

#to query another endpoint, change the URL for the service and the query
node = query_data(sparql_query)
query_value_cleaning(st)
node.to_csv("node.csv")
node.head(10)

,o,p
0,http://www.datacron-project.eu/datAcron#t_1459...,http://www.ontologydesignpatterns.org/ont/dul/...
1,http://www.datacron-project.eu/datAcron#geom_LFMT,http://www.ontologydesignpatterns.org/ont/dul/...
2,http://www.datacron-project.eu/datAcron#n_AA51...,http://www.datacron-project.eu/datAcron#precedes


TODO / PROBLEM

Why is there no weather or no altitude attached?

# Intersecting areas and trajectories (Page 5)

In [26]:
sparql_query = """ 
PREFIX : <http://www.datacron-project.eu/datAcron#>
PREFIX dul: <http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#>

SELECT ?vessel ?time ?wkt ?speed ?event ?weather 
WHERE {
    ?s1 a :Node ;
          :ofMovingObject ?vessel ;
          :hasSpeed ?speed ;
        dul:hasConstituent/:TimeStart ?time ;
        dul:hasConstituent/:hasWKT ?wkt .
FILTER(bif:st_distance( bif:st_geomfromtext ("POINT(13.139045 44.466133)"), bif:st_geomfromtext(?wkt)) <= 5 &&
    xsd:dateTime(substr(?time,1,19))<xsd:dateTime("2016-01-08 16:12:41") && 
    xsd:dateTime(substr(?time,1,19))>xsd:dateTime("2016-01-08 14:12:41"))
}
"""

#to query another endpoint, change the URL for the service and the query
ci = query_data(sparql_query)
query_value_cleaning(ci)
ci.to_csv("closeins.csv")
ci.head(10)



EndPointInternalError: EndPointInternalError: endpoint returned code 500 and response. 

Response:
b'Virtuoso S1T00 Error SR171: Transaction timed out\n\nSPARQL query:\ndefine sql:big-data-const 0 \n#output-format:application/sparql-results+json\n \nPREFIX : <http://www.datacron-project.eu/datAcron#>\nPREFIX dul: <http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#>\n\nSELECT ?vessel ?time ?wkt ?speed ?event ?weather \nWHERE {\n    ?s1 a :Node ;\n          :ofMovingObject ?vessel ;\n          :hasSpeed ?speed ;\n        dul:hasConstituent/:TimeStart ?time ;\n        dul:hasConstituent/:hasWKT ?wkt .\nFILTER(bif:st_distance( bif:st_geomfromtext ("POINT(13.139045 44.466133)"), bif:st_geomfromtext(?wkt)) <= 5 &&\n    xsd:dateTime(substr(?time,1,19))<xsd:dateTime("2016-01-08 16:12:41") && \n    xsd:dateTime(substr(?time,1,19))>xsd:dateTime("2016-01-08 14:12:41"))\n}\n'

PROBLEM

In the query above, taken directly from the tutorial, I receive a timeout on a sample query! What could be the problem here?

# Regulations rausziehen

This chapter is still under construction.


Regulations leben in einem Zeitintervall. Wenn wir also Regulations rausziehen wollen, so müssen wir immer angeben, welches Zeitintervall abgedeckt werden soll. Hierzu muss ein Startdatum $t_s$, ein Enddatum $t_e$ und die Intervallgröße $\Delta t$ angegeben werden. Die Anzahl der Timesteps ergibt sich dann als $$n = \frac{t_e - t_s}{\Delta t}$$
Es folgt: der $i-$te Zeitraum ist der, der bei $t_s + i \cdot \Delta t$ beginnt.

Wir beginnne mit einem Beispiel für den Airspace Airspace_LTBA_411, weil dieser sehr oft reguliert wird.

In [229]:
sparql_query = """ 
PREFIX : <http://www.datacron-project.eu/datAcron#>
PREFIX dul: <http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX myfn: <java:datAcronTester.unipi.gr.sparql_functions.>

SELECT (count (DISTINCT ?regulation) as ?count)
WHERE {?regulation rdf:type/rdfs:subClassOf* :FM_Regulation ;
                   dul:hasRegion 'Airspace_LTBA_411' ;
                   dul:hasTimeInterval '6' .
       ?t :TimeStart ?s ;
          :TimeEnd   ?e .
FILTER(myfn:overlaps(?s, ?e, '2016-04-01 00:00:00'^^<http://www.w3.org/2001/XMLSchema#DateTime>, 
                             '2016-04-30 23:59:59'^^<http://www.w3.org/2001/XMLSchema#DateTime>))
}
"""

df = query_data(sparql_query)
query_value_cleaning(df)
#df.to_csv("airspaceconfigs.csv")
#df = df[df.p != "#type"]
df.head(20)

,count
0,0


In [424]:
import cesiumpy

v = cesiumpy.Viewer()
b = cesiumpy.Box(dimensions=(40e4, 30e4, 50e4), material=cesiumpy.color.RED, position=[-120, 40, 0])
v.entities.add(b)

v

In [234]:
sparql_query = """ 
PREFIX : <http://www.datacron-project.eu/datAcron#>
PREFIX dul: <http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX myfn: <java:datAcronTester.unipi.gr.sparql_functions.>

SELECT ?s ?w
WHERE {?s rdf:type :Node ;
          :hasWeatherCondition ?w .
} LIMIT 1000
"""




df = query_data(sparql_query)
query_value_cleaning(df)
df.to_csv("data/someweather.csv")
#df = df[df.p != "#type"]
df.head(20)

,s,w
0,node_0_1453961518000_15.896575_43.7204233333333,weather_43.7204233333333_15.896575_0.0_1453971600
1,node_100000001_1453961006000_3.05081333333333_...,weather_36.7109833333333_3.05081333333333_0.0_...
2,node_104938_1453961970000_2.35546_51.042906666...,weather_51.0429066666667_2.35546_0.0_1453971600
3,node_10772765_1453961146000_4.71531666666667_5...,weather_51.828_4.71531666666667_0.0_1453971600
4,node_10772765_1453961746000_4.71518833333333_5...,weather_51.8279616666667_4.71518833333333_0.0_...
5,node_10772765_1453962107000_4.715225_51.827995,weather_51.827995_4.715225_0.0_1453971600
6,node_10878978_1453962062000_4.14806_51.9494466...,weather_51.9494466666667_4.14806_0.0_1453971600
7,node_10878978_1453962150000_4.14803333333333_5...,weather_51.9494333333333_4.14803333333333_0.0_...
8,node_11111111_1453961058000_-5.48814333333333_...,weather_35.8987816666667_-5.48814333333333_0.0...
9,node_11603_1453960418000_-3.03189_43.346333333...,weather_43.3463333333333_-3.03189_0.0_1453950000


In [238]:
sparql_query = """ 
PREFIX : <http://www.datacron-project.eu/datAcron#>
PREFIX dul: <http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX myfn: <java:datAcronTester.unipi.gr.sparql_functions.>

SELECT DISTINCT ?p ?o
WHERE {:weather_43.7204233333333_15.896575_0.0_1453971600  ?p        ?o .
} LIMIT 1000
"""




df = query_data(sparql_query)
query_value_cleaning(df)
df.to_csv("data/someweather_condition.csv")
#df = df[df.p != "#type"]
df.head(20)

,p,o
0,type,WeatherCondition
1,reportedDewPoint,14.859985
2,reportedMaxTemperature,14.799988
3,reportedMinTemperature,14.799988
4,reportedPressure,101688.46
5,windDirectionMax,250.32214
6,windDirectionMin,250.32214
7,windSpeedMax,1.603652
8,windSpeedMin,1.603652


In [239]:
sparql_query = """ 
PREFIX : <http://www.datacron-project.eu/datAcron#>
PREFIX dul: <http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX myfn: <java:datAcronTester.unipi.gr.sparql_functions.>

SELECT DISTINCT ?p
WHERE {?s rdf:type :WeatherCondition ; 
          ?p        ?o .
} LIMIT 1000
"""




df = query_data(sparql_query)
query_value_cleaning(df)
df.to_csv("data/someweather_condition.csv")
#df = df[df.p != "#type"]
df.head(20)

,p
0,type
1,reportedDewPoint
2,reportedMaxTemperature
3,reportedMinTemperature
4,reportedPressure
5,windDirectionMax
6,windDirectionMin
7,windSpeedMax
8,windSpeedMin


In [240]:
sparql_query = """ 
PREFIX : <http://www.datacron-project.eu/datAcron#>
PREFIX dul: <http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX myfn: <java:datAcronTester.unipi.gr.sparql_functions.>

SELECT ?s ?w ?windspeedmax
WHERE {?s rdf:type :Node ;
          :hasWeatherCondition ?w .
       ?w :windSpeedMax ?windspeedmax   
} LIMIT 1000
"""




df = query_data(sparql_query)
query_value_cleaning(df)
df.to_csv("data/someweather.csv")
#df = df[df.p != "#type"]
df.head(20)

,s,w,windspeedmax
0,node_209283000_1453961175000_-9.67183333333333...,weather_30.4155_-9.67183333333333_0.0_1453971600,3.905125
1,node_209167000_1453960982000_33.0128883333333_...,weather_34.6450183333333_33.0128883333333_0.0_...,6.0817924
2,node_209167000_1453960618000_33.01285_34.64501,weather_34.64501_33.01285_0.0_1453950000,5.7729197
3,node_209167000_1453961698000_33.012855_34.6450...,weather_34.6450216666667_33.012855_0.0_1453971600,6.0817924
4,node_209645000_1453960922000_33.0074216666667_...,weather_34.647225_33.0074216666667_0.0_1453971600,6.0817924
5,node_209645000_1453961198000_33.0074216666667_...,weather_34.647225_33.0074216666667_0.0_1453971600,6.0817924
6,node_209259000_1453961386000_33.0071833333333_...,weather_34.647385_33.0071833333333_0.0_1453971600,6.0817924
7,node_209413000_1453961126000_33.0099466666667_...,weather_34.647925_33.0099466666667_0.0_1453971600,6.0817924
8,node_209413000_1453960411000_33.009935_34.647935,weather_34.647935_33.009935_0.0_1453950000,5.7729197
9,node_209413000_1453961847000_33.009915_34.6479...,weather_34.6479466666667_33.009915_0.0_1453971600,6.0817924
